In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]=os.environ.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="ChatBot_without_RAG_or_Agents"

In [2]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-720d15a6-898b-485d-a7a9-0bdc14f97451-0')

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about you unless you share it with me. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 11, 'total_tokens': 38}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e317f68-13f9-4df0-a1cf-a0425e335676-0')

# Message History

We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. 


# Define message store


In [5]:

from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# Function to get session history
This function is expected to take in a session_id and return a Message History object. This session_id is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain


In [6]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
# Function to generate a unique session ID
import uuid

def generate_session_id() -> str:
    return str(uuid.uuid4())

In [8]:
# Generating a dynamic session ID
session_id_1 = generate_session_id()

# Define Config

We now need to create a config that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a session_id. 

In [9]:
config_1 = {"configurable": {"session_id": session_id_1}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config_1,
)


In [11]:
response

AIMessage(content='Hi Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-7b349c01-2230-4e4c-a022-9010f936cae1-0')

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config_1,
)

response

AIMessage(content='Your name is Bob! How can I help you today, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 33, 'total_tokens': 47}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e4adbe8-4cfc-49d8-9390-9ee34b195d20-0')

# Generate a new session to demo that it won't have the context

In [13]:
session_id_2 = generate_session_id()

In [14]:
config_2 = {"configurable": {"session_id": session_id_2}}

In [15]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config_2,
)

response

AIMessage(content="I'm sorry, but I don't have access to personal information about you unless you share it with me. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 11, 'total_tokens': 38}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-bf338013-3601-4ca4-8386-5f6dcadbd9ed-0')

# The first session would still have the context

In [16]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config_1,
)

response

AIMessage(content="Your name is Bob. If there's anything else you'd like to talk about or ask, feel free!", response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 59, 'total_tokens': 79}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None}, id='run-b481448a-4758-4b7f-82dc-1908eb6f5e7c-0')